<a href="https://colab.research.google.com/github/JDittes/Amazon_Vine_Analysis/blob/main/Vine_Review_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
# Find the latest version of spark 3.0 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.0.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:3 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [696 B]
Hit:6 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:8 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Get:9 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:11 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ Packages [82.3 kB]
Hit:12 http://ppa.launchpad.net/cran/

In [2]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2022-04-09 20:17:05--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar’

postgresql-42.2.16. 100%[===================>] 979.38K  --.-KB/s    in 0.08s   

2022-04-09 20:17:06 (11.7 MB/s) - ‘postgresql-42.2.16.jar’ saved [1002883/1002883]



In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("M16-Amazon-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

### Load Amazon Data into Spark DataFrame

In [4]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Watches_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get(""), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|    3653882|R3O9SGZBVQBV76|B00FALQ1ZC|     937001370|Invicta Women's 1...|         Watches|          5|            0|          0|   N|                Y|          Five Stars|Absolutely love t...| 2015-08-31|
|         US|   14661224| RKH8BNC3L5DLF|B00D3RGO20|     484010722|Kenneth Cole New ...|         Watches|          5|    

In [5]:
# Configure settings for RDS
mode = "append"
jdbc_url="jdbc:postgresql://amz-book-revs.czwz82lmzz0z.us-east-2.rds.amazonaws.com:5432/postgres"
config = {"user":"postgres", 
          "password": "1chMagBucher", 
          "driver":"org.postgresql.Driver"}

In [6]:
# Create the vine_table. DataFrame
vine_table = df.select('review_id','star_rating','helpful_votes','total_votes','vine','verified_purchase')
vine_table.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R3O9SGZBVQBV76|          5|            0|          0|   N|                Y|
| RKH8BNC3L5DLF|          5|            0|          0|   N|                Y|
|R2HLE8WKZSU3NL|          2|            1|          1|   N|                Y|
|R31U3UH5AZ42LL|          5|            0|          0|   N|                Y|
|R2SV659OUJ945Y|          4|            0|          0|   N|                Y|
| RA51CP8TR5A2L|          5|            0|          0|   N|                Y|
| RB2Q7DLDN6TH6|          5|            1|          1|   N|                Y|
|R2RHFJV0UYBK3Y|          1|            1|          5|   N|                N|
|R2Z6JOQ94LFHEP|          5|            1|          2|   N|                Y|
| RX27XIIWY5JPB|          4|            0|          0|   N|     

In [7]:
from pyspark.sql.types import IntegerType
total_votes_df = vine_table.withColumn('total_votes', vine_table['total_votes'].cast(IntegerType()))


In [8]:
total_votes_df = total_votes_df.withColumn('helpful_votes', total_votes_df['helpful_votes'].cast(IntegerType()))

In [9]:
total_votes_df.dropna()

DataFrame[review_id: string, star_rating: int, helpful_votes: int, total_votes: int, vine: string, verified_purchase: string]

In [10]:
from pyspark.sql.functions import col
total_votes_df = total_votes_df.filter(col('total_votes') >= 20)
total_votes_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R14W2VCHHK5V7W|          5|           19|         20|   N|                Y|
|R1S3T57O3OYT5S|          5|           19|         20|   N|                Y|
|R1BTWIBLYYVOV7|          5|           30|         30|   N|                Y|
| R6F9VY91ADPLA|          1|            8|         30|   N|                N|
|R3PXNV89DFIXKV|          5|           35|         37|   N|                Y|
|R2ZF9NYVT3J7D6|          5|           19|         22|   N|                Y|
|R20NYA5V0UF9NE|          5|           27|         28|   N|                Y|
|R2X8FZRUOS8R8C|          4|           25|         26|   N|                Y|
|R2D8IMBVX3XCLF|          1|           14|         20|   N|                Y|
|R25UD9TA63L3Q8|          5|           25|         27|   N|     

In [11]:
votes_div = total_votes_df['helpful_votes']/total_votes_df['total_votes']
total_votes_df = total_votes_df.withColumn('Ratio', votes_div)
total_votes_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+-------------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|              Ratio|
+--------------+-----------+-------------+-----------+----+-----------------+-------------------+
|R14W2VCHHK5V7W|          5|           19|         20|   N|                Y|               0.95|
|R1S3T57O3OYT5S|          5|           19|         20|   N|                Y|               0.95|
|R1BTWIBLYYVOV7|          5|           30|         30|   N|                Y|                1.0|
| R6F9VY91ADPLA|          1|            8|         30|   N|                N|0.26666666666666666|
|R3PXNV89DFIXKV|          5|           35|         37|   N|                Y| 0.9459459459459459|
|R2ZF9NYVT3J7D6|          5|           19|         22|   N|                Y| 0.8636363636363636|
|R20NYA5V0UF9NE|          5|           27|         28|   N|                Y| 0.9642857142857143|
|R2X8FZRUOS8R8C|    

In [12]:
helpful_votes_df = total_votes_df.filter('Ratio >= 0.5')
helpful_votes_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+------------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|             Ratio|
+--------------+-----------+-------------+-----------+----+-----------------+------------------+
|R14W2VCHHK5V7W|          5|           19|         20|   N|                Y|              0.95|
|R1S3T57O3OYT5S|          5|           19|         20|   N|                Y|              0.95|
|R1BTWIBLYYVOV7|          5|           30|         30|   N|                Y|               1.0|
|R3PXNV89DFIXKV|          5|           35|         37|   N|                Y|0.9459459459459459|
|R2ZF9NYVT3J7D6|          5|           19|         22|   N|                Y|0.8636363636363636|
|R20NYA5V0UF9NE|          5|           27|         28|   N|                Y|0.9642857142857143|
|R2X8FZRUOS8R8C|          4|           25|         26|   N|                Y|0.9615384615384616|
|R2D8IMBVX3XCLF|          1|  

In [42]:
total_fives = helpful_votes_df.filter('star_rating == 5').count()
total_fives

4347

In [13]:
vine_yviews_df = helpful_votes_df.filter(col("vine").contains('Y'))
vine_yviews_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+------------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|             Ratio|
+--------------+-----------+-------------+-----------+----+-----------------+------------------+
|R1B7M0OP3UNP6O|          5|           49|         52|   Y|                N|0.9423076923076923|
|R2UUV4UGGYMQG8|          5|           34|         39|   Y|                N|0.8717948717948718|
| R9K0LZV2BK9YY|          4|           37|         39|   Y|                N|0.9487179487179487|
|R2OVFLNEUEGTJM|          3|           18|         25|   Y|                N|              0.72|
| RBE09ELJ77LQ0|          5|           44|         45|   Y|                N|0.9777777777777777|
|R3867T8AIJJHM6|          5|           26|         27|   Y|                N|0.9629629629629629|
|R1FNVUXPU63WOZ|          4|           43|         48|   Y|                N|0.8958333333333334|
|R25XGG2G12SE1Z|          4|  

In [14]:
vine_nviews_df = helpful_votes_df.filter(col("vine").contains('N'))
vine_nviews_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+------------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|             Ratio|
+--------------+-----------+-------------+-----------+----+-----------------+------------------+
|R14W2VCHHK5V7W|          5|           19|         20|   N|                Y|              0.95|
|R1S3T57O3OYT5S|          5|           19|         20|   N|                Y|              0.95|
|R1BTWIBLYYVOV7|          5|           30|         30|   N|                Y|               1.0|
|R3PXNV89DFIXKV|          5|           35|         37|   N|                Y|0.9459459459459459|
|R2ZF9NYVT3J7D6|          5|           19|         22|   N|                Y|0.8636363636363636|
|R20NYA5V0UF9NE|          5|           27|         28|   N|                Y|0.9642857142857143|
|R2X8FZRUOS8R8C|          4|           25|         26|   N|                Y|0.9615384615384616|
|R2D8IMBVX3XCLF|          1|  

In [15]:
helpful_votes_df.agg({'total_votes': 'sum'}).show()
#https://www.geeksforgeeks.org/how-to-find-the-sum-of-particular-column-in-pyspark-dataframe/

+----------------+
|sum(total_votes)|
+----------------+
|          396475|
+----------------+



In [43]:
review_count = helpful_votes_df.count()
print(f'The total number of reviews is {review_count}')
print(f'The total number of 5-star reviews is {total_fives}')


The total number of reviews is 8409
The total number of 5-star reviews is 4347


In [24]:
vine_yviews_df.filter(col("star_rating").contains('5')).show()


+--------------+-----------+-------------+-----------+----+-----------------+------------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|             Ratio|
+--------------+-----------+-------------+-----------+----+-----------------+------------------+
|R1B7M0OP3UNP6O|          5|           49|         52|   Y|                N|0.9423076923076923|
|R2UUV4UGGYMQG8|          5|           34|         39|   Y|                N|0.8717948717948718|
| RBE09ELJ77LQ0|          5|           44|         45|   Y|                N|0.9777777777777777|
|R3867T8AIJJHM6|          5|           26|         27|   Y|                N|0.9629629629629629|
|R2PQYOCJXRB1BF|          5|           26|         28|   Y|                N|0.9285714285714286|
|R2OFG89WKPLHGU|          5|           54|         60|   Y|                N|               0.9|
|R1YBNRRUD5WCG4|          5|           15|         20|   Y|                N|              0.75|
|R33Y1P0VTKF46V|          5|  

In [45]:
five_star_y = vine_yviews_df.filter(col("star_rating").contains('5'))
five_star_count = five_star_y.count()
five_star_count
pct_y = (five_star_count/total_fives) * 100
print(f'{pct_y} percent of 5-star reviews are from the Vine program')

0.3450655624568668 percent of 5-star reviews are from the Vine program


In [18]:
count_fives_df = vine_nviews_df.filter(col("star_rating").contains('5'))
count_fives_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+------------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|             Ratio|
+--------------+-----------+-------------+-----------+----+-----------------+------------------+
|R14W2VCHHK5V7W|          5|           19|         20|   N|                Y|              0.95|
|R1S3T57O3OYT5S|          5|           19|         20|   N|                Y|              0.95|
|R1BTWIBLYYVOV7|          5|           30|         30|   N|                Y|               1.0|
|R3PXNV89DFIXKV|          5|           35|         37|   N|                Y|0.9459459459459459|
|R2ZF9NYVT3J7D6|          5|           19|         22|   N|                Y|0.8636363636363636|
|R20NYA5V0UF9NE|          5|           27|         28|   N|                Y|0.9642857142857143|
|R25UD9TA63L3Q8|          5|           25|         27|   N|                Y|0.9259259259259259|
|R2RB1HML8N712P|          5|  

In [47]:
five_starn_count = count_fives_df.count()
five_starn_count
pct_n = (five_starn_count/total_fives) * 100
print(f'{pct_n} percent of 5-star reviews are NOT from the Vine program')

99.65493443754313 percent of 5-star reviews are NOT from the Vine program


In [19]:
count_fives_df.count()
print(f'There are {count_fives_df.count()} 5-star Reviews')

There are 4332 5-star Reviews


## Practice exporting to Pandas

In [20]:
counts_df = count_fives_df.toPandas()

In [21]:
counts_df.head()

,review_id,star_rating,helpful_votes,total_votes,vine,verified_purchase,Ratio
0,R14W2VCHHK5V7W,5,19,20,N,Y,0.950000
1,R1S3T57O3OYT5S,5,19,20,N,Y,0.950000
2,R1BTWIBLYYVOV7,5,30,30,N,Y,1.000000
3,R3PXNV89DFIXKV,5,35,37,N,Y,0.945946
4,R2ZF9NYVT3J7D6,5,19,22,N,Y,0.863636
